In [2]:
import numpy as np
import pandas as pd
import get_data
from scipy.spatial.distance import cdist

In [3]:
def get_top_vineyard(desired, lands, k=3):
    desired = np.array([np.array(desired, dtype=float)])
    lands = np.array(lands, dtype=float)
    distances = cdist(desired, lands)[0]
    closest_lands = lands[np.argpartition(distances, k)[:k]]
    distances = cdist(desired, closest_lands)[0]
    return np.argsort(distances)

In [4]:
vine = pd.read_csv("data.csv")

In [5]:
lands = pd.read_csv("lands.csv")

In [6]:
vine.iloc[0]

x                          197
y                         2445
admin                    False
landcover                    7
relief_aspect            360.0
relief_height               17
relief_slope          1.432096
relief_soiltexture           3
sunny_days                 111
vineyard                  True
water_seasonality            0
Name: 0, dtype: object

In [14]:
lands.iloc[0]

x                         497
y                         651
admin                   False
landcover                   0
relief_aspect        -99999.0
relief_height               0
relief_slope              0.0
relief_soiltexture          0
sunny_days                147
vineyard                False
water_seasonality          12
Name: 0, dtype: object

In [7]:
np.array(vine.iloc[0], dtype=float)

array([1.9700000e+02, 2.4450000e+03, 0.0000000e+00, 7.0000000e+00,
       3.6000000e+02, 1.7000000e+01, 1.4320962e+00, 3.0000000e+00,
       1.1100000e+02, 1.0000000e+00, 0.0000000e+00])

In [8]:
get_top_vineyard(vine.iloc[0], lands)

array([1, 0, 2])

In [31]:
def find_closest(wineyards_x, wineyards_y, k):
  wineyards = pd.read_csv("data.csv")
  lands = pd.read_csv("lands.csv")
  desired_wineyards = pd.DataFrame(columns=lands.columns)
  for i in range(len(wineyards_x)):
    s = wineyards.where(wineyards.x == wineyards_x[i]).where(wineyards.y == wineyards_y[i]).iloc[0]
    desired_wineyards = desired_wineyards.append(s)
  # print(desired_wineyards)
  des = desired_wineyards.mean()
  # print(des)
  needed_indices = get_top_vineyard(des, lands, k)
  ans_x = []
  ans_y = []
  for i in range(k):
    ans_x.append(lands.iloc[needed_indices[i]].x)
    ans_y.append(lands.iloc[needed_indices[i]].y)
  return ans_x, ans_y

In [32]:
x, y = find_closest([197], [2445], 5)